Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import ast
import datetime
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import norm

warnings.filterwarnings("ignore")

Merge Movies and Ratings

In [2]:
ratings = pd.read_csv('moviesRatings/ratings.csv')
moviesMetadata = pd.read_csv('moviesRatings/movies_metadata.csv')
credits = pd.read_csv('moviesRatings/credits.csv')

# Creating functions to extract only cast name and director names from respective columns
# Converting the 'cast' column such that it only contains an array of the actors
def cast_convert(x):
    cast_names = []
    for i in ast.literal_eval(x):
        cast_names.append(i['name'])
    return cast_names

# Converting the 'crew' column such that it only contains an array of the director
def director_convert(x):
    director_names=[]
    for i in ast.literal_eval(x):
        if i['job']=='Director':
            director_names.append(i['name'])
        else:
            continue
    return director_names

In [4]:
credits['cast']=credits['cast'].apply(cast_convert) # Applying the director_convert function
credits['director']=credits['crew'].apply(director_convert) # Applying the cast_convert function
cast_dir = credits[['id','director','cast']] # Create new dataframe with movie_id, director and cast
cast_dir = cast_dir[cast_dir['cast'].map(lambda d: len(d)) > 0] # Removing rows where there are empty lists
# change id dtype to int
moviesMetadata['id'] = pd.to_numeric(moviesMetadata['id'], errors='coerce').astype('Int64')

moviesMetadata = moviesMetadata.dropna(subset=["id"], axis=0) # drop rows with id == NA
merge_ratings = ratings.merge( moviesMetadata, left_on='movieId', right_on='id', how='inner') # inner join on movieID
merge_ratings = merge_ratings.drop(columns=['movieId', 'belongs_to_collection', 'homepage', 'poster_path', 'timestamp', 'video', 'status', 'original_title'], index=1)# drop some columns 
merge_ratings = merge_ratings[merge_ratings['userId'].isin(merge_ratings["userId"].value_counts()[merge_ratings["userId"].value_counts() > 500].index)] # filter for users with more than 500 views

#convert to the right data type
merge_ratings["adult"] = merge_ratings["adult"].astype(bool)
merge_ratings['release_date'] = pd.to_datetime(merge_ratings['release_date'])
merge_ratings['genres'] = merge_ratings['genres'].apply(ast.literal_eval)
merge_ratings['production_companies'] = merge_ratings['production_companies'].apply(ast.literal_eval)
merge_ratings['production_countries'] = merge_ratings['production_countries'].apply(ast.literal_eval)
merge_ratings['spoken_languages'] = merge_ratings['spoken_languages'].apply(ast.literal_eval)

#convert the dictionaries into arrays of ids
merge_ratings['production_countries_name'] = merge_ratings['production_countries'].apply(lambda x : [item["iso_3166_1"] for item in x] if isinstance(x, list) else [])
merge_ratings['spoken_languages_name'] = merge_ratings['spoken_languages'].apply(lambda x : [item["iso_639_1"] for item in x] if isinstance(x, list) else [])

merge_ratings['production_companies_name'] = merge_ratings['production_companies'].apply(lambda x : [item["name"] for item in x] if isinstance(x, list) else [])
merge_ratings['genre_name'] = merge_ratings['genres'].apply(lambda x : [item["name"] for item in x] if isinstance(x, list) else [])

# Merge Movies, Ratings, Keywords

Extract keywords IDs as a new column and remove NAs

In [5]:
keywords = pd.read_csv('moviesRatings/keywords.csv') # Contains the movie plot keywords for our MovieLens movies collected from the TMDB 
keywords # id corresponds to movieID
cast_kw = pd.merge(cast_dir, keywords, on="id", how="left")
merge_kw = merge_ratings.merge(cast_kw, left_on="id", right_on="id", how="left")

print(f"""
intersection   {len(set(merge_ratings["id"]).intersection(set(keywords["id"])))}
ratings+movies {len(set(merge_ratings["id"]))}
keywords       {len(set(keywords["id"]))}
""")

merge_kw["keywords"].value_counts(dropna=False) # IDs in keywords are all present in ratings + movies df
merge_kw = merge_kw.dropna(subset=["keywords"], axis=0) 
merge_kw = merge_kw[merge_kw["keywords"] != "[]"].reset_index(drop=True) # drop rows without any keywords

def extract_ids(json_string):
    lst_of_ids = []
    lst_of_kw = ast.literal_eval(json_string)
    for d in lst_of_kw:
        id = d["id"]
        lst_of_ids.append(id)
    return lst_of_ids 

merge_kw['keywords_id'] = merge_kw.apply(lambda row: extract_ids(row["keywords"]), axis=1) # get new column, each row contains list of the movie's keywords IDs


intersection   7063
ratings+movies 7063
keywords       45432



,keywords,keywords_id
0,"[{'id': 934, 'name': 'judge'}, {'id': 1533, 'n...","[934, 1533, 2863, 4918, 5259, 5260, 9935, 15162]"
1,"[{'id': 934, 'name': 'judge'}, {'id': 1533, 'n...","[934, 1533, 2863, 4918, 5259, 5260, 9935, 15162]"
2,"[{'id': 934, 'name': 'judge'}, {'id': 1533, 'n...","[934, 1533, 2863, 4918, 5259, 5260, 9935, 15162]"
3,"[{'id': 934, 'name': 'judge'}, {'id': 1533, 'n...","[934, 1533, 2863, 4918, 5259, 5260, 9935, 15162]"
4,"[{'id': 934, 'name': 'judge'}, {'id': 1533, 'n...","[934, 1533, 2863, 4918, 5259, 5260, 9935, 15162]"
...,...,...
775442,"[{'id': 10768, 'name': 'lesbian'}, {'id': 1870...","[10768, 187056, 208508]"
775443,"[{'id': 187056, 'name': 'woman director'}]",[187056]
775444,"[{'id': 154802, 'name': 'silent film'}]",[154802]
775445,"[{'id': 849, 'name': 'dc comics'}, {'id': 9715...","[849, 9715, 11134, 163455, 179991]"


Merge Movies, Ratings, Keywords, Sentiment Scores

In [6]:
sentiment = pd.read_csv("moviesRatings/reddit_sentiment_scores.csv")
result = pd.merge(merge_kw, sentiment, left_on="title", right_on="movie_title", how="left")
result = result.dropna(subset="average_sentiment")# drop rows without sentiment score
#result.to_csv("merged_dfs.csv", index=False)